In [1]:
from amadeus import Client, ResponseError
amadeus = Client(
        client_id='CGwOmHn7cmfAIuUcbqUiaPC5LAyAvwAG',
        client_secret='rKvILHDsjxcCh6yq'
    )

In [2]:
try:
    firstResponse = amadeus.shopping.flight_offers.prediction.post(
        amadeus.shopping.flight_offers_search.get(
        originLocationCode='SYD',
        destinationLocationCode='BKK',
        departureDate='2022-11-01',
        adults=1).result)
except ResponseError as error:
    print(error)

In [12]:
try:
    secondResponse = amadeus.shopping.flight_offers_search.get(
    originLocationCode='MAD',
    destinationLocationCode='BOS',
    departureDate='2022-11-01',
    adults='4'
    )
except ResponseError as error:
    print(error)

In [13]:
print(secondResponse.data)

[{'type': 'flight-offer', 'id': '1', 'source': 'GDS', 'instantTicketingRequired': False, 'nonHomogeneous': False, 'oneWay': False, 'lastTicketingDate': '2022-06-13', 'numberOfBookableSeats': 9, 'itineraries': [{'duration': 'PT10H', 'segments': [{'departure': {'iataCode': 'MAD', 'terminal': '2', 'at': '2022-11-01T10:20:00'}, 'arrival': {'iataCode': 'LIS', 'terminal': '1', 'at': '2022-11-01T10:40:00'}, 'carrierCode': 'TP', 'number': '1011', 'aircraft': {'code': 'E90'}, 'operating': {'carrierCode': 'NI'}, 'duration': 'PT1H20M', 'id': '1', 'numberOfStops': 0, 'blacklistedInEU': False}, {'departure': {'iataCode': 'LIS', 'terminal': '1', 'at': '2022-11-01T11:40:00'}, 'arrival': {'iataCode': 'BOS', 'terminal': 'E', 'at': '2022-11-01T15:20:00'}, 'carrierCode': 'TP', 'number': '217', 'aircraft': {'code': '32Q'}, 'operating': {'carrierCode': 'TP'}, 'duration': 'PT7H40M', 'id': '2', 'numberOfStops': 0, 'blacklistedInEU': False}]}], 'price': {'currency': 'EUR', 'total': '1187.64', 'base': '588.00'

In [49]:
class temp:
    amadeus = Client(
        client_id='CGwOmHn7cmfAIuUcbqUiaPC5LAyAvwAG',
        client_secret='rKvILHDsjxcCh6yq'
    )
    def extractDuration(self, itineraries):
        hour = 0
        minute = 0
        for i in itineraries:
            i = {x.replace(' ', ''): v
    for x, v in i.items()}
            currDur = i['duration'].split('M')
            if (len(currDur) == 1):
                hour += int(currDur[-1].split('PT')[-1].split('H')[0])
            if (len(currDur) == 2):
                minute += int(currDur[0].split('H')[-1])
                hour += int(currDur[0].split('H')[0].split('PT')[-1])
        return (hour, minute)
    def extractPrice(self, price):
        price = {x.replace(' ', ''): v
             for x, v in price.items()}
        return (price['currency'], price['total'])
    def extractItinerary(self, itineraries):
        return itineraries
    def flightEndingTimeArray(self, response):
        resArr = []
        for i in response['itineraries'][0]['segments']:
            resArr.append(i['arrival']['at'])
        return resArr
    def flightDestinationLocationArray(self, response):
        resArr = []
        for i in response['itineraries'][0]['segments']:
            resArr.append({'airport' : i['arrival']['iataCode'],  'terminal' : i['arrival']['terminal']})
        return resArr
    def flightStartingTimeArray(self, response):
        resArr = []
        for i in response['itineraries'][0]['segments']:
            resArr.append(i['departure']['at'])
        return resArr
    def flightSourceLocationArray(self, response):
        resArr = []
        for i in response['itineraries'][0]['segments']:
            resArr.append({'airport' : i['departure']['iataCode'], 'tיerminal' : i['departure']['terminal']})
        return resArr
    def parseFlightResponse(self, response):
        resultArr = []
        for option in response:
            currResultDict = {}
            print(option)
            print(option.keys())
            option = {x.replace(' ', ''): v
     for x, v in option.items()}
            print("Duration", self.extractDuration(option['itineraries']))
            print("Price", self.extractPrice(option['price']))
            print("Itinerary:", self.extractItinerary(option['itineraries']))
            print("Time start:", self.flightStartingTimeArray(option))
            print("Source location:", self.flightSourceLocationArray(option))
            print("Time end:", self.flightEndingTimeArray(option))
            print("Destination location:", self.flightDestinationLocationArray(option))
            break
            #currResultDict['duration'] = self.extractDuration(option['itineraries'])
            #self.extractInfo(option[0])


In [50]:
a = temp()
a.parseFlightResponse(secondResponse.data)

{'type': 'flight-offer', 'id': '1', 'source': 'GDS', 'instantTicketingRequired': False, 'nonHomogeneous': False, 'oneWay': False, 'lastTicketingDate': '2022-06-13', 'numberOfBookableSeats': 9, 'itineraries': [{'duration': 'PT10H', 'segments': [{'departure': {'iataCode': 'MAD', 'terminal': '2', 'at': '2022-11-01T10:20:00'}, 'arrival': {'iataCode': 'LIS', 'terminal': '1', 'at': '2022-11-01T10:40:00'}, 'carrierCode': 'TP', 'number': '1011', 'aircraft': {'code': 'E90'}, 'operating': {'carrierCode': 'NI'}, 'duration': 'PT1H20M', 'id': '1', 'numberOfStops': 0, 'blacklistedInEU': False}, {'departure': {'iataCode': 'LIS', 'terminal': '1', 'at': '2022-11-01T11:40:00'}, 'arrival': {'iataCode': 'BOS', 'terminal': 'E', 'at': '2022-11-01T15:20:00'}, 'carrierCode': 'TP', 'number': '217', 'aircraft': {'code': '32Q'}, 'operating': {'carrierCode': 'TP'}, 'duration': 'PT7H40M', 'id': '2', 'numberOfStops': 0, 'blacklistedInEU': False}]}], 'price': {'currency': 'EUR', 'total': '1187.64', 'base': '588.00',

In [19]:
print(secondResponse.data[0])
print(secondResponse.data[0].keys())

{'type': 'flight-offer', 'id': '1', 'source': 'GDS', 'instantTicketingRequired': False, 'nonHomogeneous': False, 'oneWay': False, 'lastTicketingDate': '2022-06-13', 'numberOfBookableSeats': 9, 'itineraries': [{'duration': 'PT10H', 'segments': [{'departure': {'iataCode': 'MAD', 'terminal': '2', 'at': '2022-11-01T10:20:00'}, 'arrival': {'iataCode': 'LIS', 'terminal': '1', 'at': '2022-11-01T10:40:00'}, 'carrierCode': 'TP', 'number': '1011', 'aircraft': {'code': 'E90'}, 'operating': {'carrierCode': 'NI'}, 'duration': 'PT1H20M', 'id': '1', 'numberOfStops': 0, 'blacklistedInEU': False}, {'departure': {'iataCode': 'LIS', 'terminal': '1', 'at': '2022-11-01T11:40:00'}, 'arrival': {'iataCode': 'BOS', 'terminal': 'E', 'at': '2022-11-01T15:20:00'}, 'carrierCode': 'TP', 'number': '217', 'aircraft': {'code': '32Q'}, 'operating': {'carrierCode': 'TP'}, 'duration': 'PT7H40M', 'id': '2', 'numberOfStops': 0, 'blacklistedInEU': False}]}], 'price': {'currency': 'EUR', 'total': '1187.64', 'base': '588.00',

In [27]:
# Flight Create Orders
amadeus.booking.flight_orders.post(secondResponse.data[0], ['1'])

ClientError: [400]


In [ ]:
print("DONE")
# Flight Order Management
# The flight ID comes from the Flight Create Orders (in test environment it's temporary)
# Retrieve the order based on it's ID
flight_booking = amadeus.booking.flight_orders.post(body).data
amadeus.booking.flight_order(flight_booking['id']).get()
# Delete the order based on it's ID
amadeus.booking.flight_order(flight_booking['id']).delete()